In [ ]:
# Install required packages
!pip install transformers torch langchain faiss-cpu sentence-transformers numpy
!pip install -U langchain-community
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 94.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 72.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 24.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [ ]:
# Import libraries
from google.colab import drive
import torch
import numpy as np
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from transformers import AutoTokenizer, AutoModelForCausalLM # Make sure this line is present
import faiss

# Paths to your files in Google Drive
model_path = "/content/drive/MyDrive/phi_finetuned"  # Update if
# model_name = "microsoft/phi-3-mini-4k-instruct"
srs_file_path = "doc.txt"  # Update if needed

print("Setup Complete!")

Setup Complete!


In [ ]:
# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path, torch_dtype=torch.float16)
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# Load embedding model for RAG
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Function to create FAISS vector store from SRS document
def create_vector_store(srs_text):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=50)
    chunks = text_splitter.split_text(srs_text)
    vectors = embedding_model.embed_documents(chunks)
    vectors = np.array(vectors, dtype=np.float32)
    index = faiss.IndexFlatL2(vectors.shape[1])
    index.add(vectors)
    faiss.write_index(index, "srs_faiss.index")
    return index, chunks

# Load SRS Document
with open(srs_file_path, "r", encoding="utf-8") as file:
    srs_text = file.read()

# Create FAISS vector store
index, chunks = create_vector_store(srs_text)
print("SRS Document Indexed for RAG!")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

<ipython-input-3-40c39fe4fb8e>:8: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models 

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

SRS Document Indexed for RAG!


In [ ]:
# Function to retrieve relevant chunks
def retrieve_relevant_chunks(query, top_k=3):
    query_embedding = np.array(embedding_model.embed_query(query)).reshape(1, -1)
    index = faiss.read_index("srs_faiss.index")
    distances, indices = index.search(query_embedding, top_k)
    retrieved_docs = [chunks[i] for i in indices[0]]
    return " ".join(retrieved_docs)

# Function to generate test cases
def generate_test_cases(srs_query, max_length=1000):
    context = retrieve_relevant_chunks(srs_query)
    prompt = f"Generate test cases based on the following with detail explanation:\n{context}\nTest Cases:\n"
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    output = model.generate(**inputs, max_length=max_length, do_sample=True, top_k=50, temperature=0.7)
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    # Extract test cases after "Test Cases:"
    test_cases_start = response.find("Test Cases:")
    if test_cases_start != -1:
        test_cases = response[test_cases_start + len("Test Cases:"):].strip()
        return test_cases, prompt  # Return prompt for RL later
    return response, prompt  # Fallback

# Example Query
query = "Generate test cases for Functional Requirements"
test_cases, prompt = generate_test_cases(query)
print("Initial Test Cases:\n", test_cases)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Initial Test Cases:
 - Test Case 1: User Registration
  - Step 1: Navigate to the OSS home page.
  - Step 2: Click on the "Register" button.
  - Step 3: Enter a valid email and password.
  - Step 4: Verify that a confirmation email is sent to the email address.
  - Step 5: Verify that the user can log in with the email and password.
  - Expected Result: The user can create an account and log in successfully.
- Test Case 2: User Login
  - Step 1: Navigate to the OSS home page.
  - Step 2: Click on the "Login" button.
  - Step 3: Enter a valid email and password.
  - Step 4: Verify that the user is redirected to the product page.
  - Step 5: Verify that the user can browse and search for products.
  - Expected Result: The user can login and access the product page.
- Test Case 3: Search Functionality
  - Step 1: Navigate to the OSS home page.
  - Step 2: Enter a valid search query in the search box.
  - Step 3: Click on the "Search" button.
  - Step 4: Verify that the search results are 

In [ ]:
import torch

# RL Agent class to handle feedback and model updates
class RLAgent:
    def __init__(self, model, tokenizer):
        self.model = model
        self.tokenizer = tokenizer
        self.optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)  # Small learning rate for fine-tuning

    def get_reward(self, test_case):
        print(f"Generated Test Case:\n{test_case}")
        while True:
            try:
                feedback = int(input("Enter feedback (1=good, 0=neutral, -1=bad): "))
                if feedback in [1, 0, -1]:
                    return feedback  # Should return int
                print("Invalid input. Please enter 1, 0, or -1.")
            except ValueError:
                print("Invalid input. Please enter an integer (1, 0, or -1).")

    def update_model(self, prompt, reward):
        # Tokenize the prompt with truncation to save memory
        inputs = self.tokenizer(prompt, return_tensors="pt", max_length=512, truncation=True).to(device)
        outputs = self.model(**inputs, labels=inputs["input_ids"])
        loss = outputs.loss

        # Adjust loss based on reward (-1, 0, 1 scale)
        adjusted_loss = -reward * loss  # Negative to maximize reward for positive feedback
        self.optimizer.zero_grad()
        adjusted_loss.backward()
        self.optimizer.step()

        # Clear gradients and cache
        self.optimizer.zero_grad()
        torch.cuda.empty_cache()

# Initialize RL Agent
rl_agent = RLAgent(model, tokenizer)
print("RL Agent Initialized!")

RL Agent Initialized!


In [ ]:
import torch
import os
import gc

# Set PyTorch CUDA allocator configuration to avoid fragmentation
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# Feedback loop generating 10 types of test cases
def feedback_loop(initial_test_cases, initial_prompt, iterations=10):
    current_test_cases = initial_test_cases
    base_prompt = initial_prompt  # SRS context
    previous_feedback = ""  # Store feedback history

    # Define 10 distinct test case types
    test_case_types = [
        "Functional Test Cases",
        "Edge Case Test Cases",
        "Performance Test Cases",
        "Usability Test Cases",
        "Security Test Cases",
        "Compatibility Test Cases",
        "Regression Test Cases",
        "Integration Test Cases",
        "Negative Test Cases",
        "Exploratory Test Cases"
    ]

    for iteration in range(iterations):
        test_type = test_case_types[iteration]
        print(f"\nFeedback Round {iteration + 1} of {iterations} - Generating {test_type}:")

        # Generate test cases for the specific type
        with torch.no_grad():
            prompt = (f"{base_prompt}\nPrevious Feedback: {previous_feedback}\n"
                      f"Generate {test_type}:\nTest Cases:\n")
            current_test_cases, _ = generate_test_cases(query, max_length=300)
        print(f"{test_type}:\n{current_test_cases}")

        # Collect feedback
        reward = rl_agent.get_reward(current_test_cases)
        print(f"Reward received: {reward}")

        # Update model based on feedback
        torch.cuda.empty_cache()
        try:
            rl_agent.update_model(prompt, reward)
            print("Model updated based on feedback.")
        except RuntimeError as e:
            print(f"Memory error during update: {e}")
            torch.cuda.empty_cache()
            continue

        # Handle feedback to influence next iteration
        if reward == 1:
            print(f"Feedback is good (1) for {test_type}. Proceeding.")
            previous_feedback = f"Good (1) for {test_type}"
        elif reward == 0:
            print(f"Feedback is normal (0) for {test_type}. Modifying for next type.")
            previous_feedback = f"Normal (0) for {test_type} - Improve next time."
        elif reward == -1:
            feedback_text = input(f"Provide feedback text to improve {test_type}: ")
            previous_feedback = f"Bad (-1) for {test_type} - {feedback_text}"
            print("Feedback recorded for next iteration.")
        else:
            print(f"Unexpected reward value: {reward}. Continuing...")
            previous_feedback = f"Unexpected ({reward}) for {test_type}"

        # Minimal memory cleanup
        torch.cuda.empty_cache()

    print(f"\nCompleted {iterations} iterations. Model updated for 10 test case types.")
    return current_test_cases

# Run feedback loop
initial_test_cases = test_cases  # From Part 3
initial_prompt = prompt  # From Part 3
refined_test_cases = feedback_loop(initial_test_cases, initial_prompt)
print("\nFinal Refined Test Cases (Last Type):\n", refined_test_cases)

# Save the fine-tuned model
torch.cuda.empty_cache()
try:
    model.save_pretrained("/content/drive/MyDrive/phi_finetuned1")
    tokenizer.save_pretrained("/content/drive/MyDrive/phi_finetuned1")
    print("Fine-tuned model saved successfully.")
except RuntimeError as e:
    print(f"Error saving model: {e}")
finally:
    torch.cuda.empty_cache()

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Feedback Round 1 of 10 - Generating Functional Test Cases:
Functional Test Cases:
- Test Case ID: TC-01
- Test Case Name: Verify user registration
- Test Steps:
  1. Navigate to the registration page on the OSS homepage.
  2. Enter a valid email and password in the respective fields.
  3. Click on the
Generated Test Case:
- Test Case ID: TC-01
- Test Case Name: Verify user registration
- Test Steps:
  1. Navigate to the registration page on the OSS homepage.
  2. Enter a valid email and password in the respective fields.
  3. Click on the
Enter feedback (1=good, 0=neutral, -1=bad): 0
Reward received: 0


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Memory error during update: CUDA out of memory. Tried to allocate 50.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 30.12 MiB is free. Process 184002 has 14.71 GiB memory in use. Of the allocated memory 13.84 GiB is allocated by PyTorch, and 751.25 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

Feedback Round 2 of 10 - Generating Edge Case Test Cases:
Edge Case Test Cases:
| Test Case ID | Test Case Description | Test Steps | Expected Result | Actual Result | Pass/Fail |
| --- | --- | --- | --- | --- | --- |
| TC-001 | User Registration | 
  - Navigate to the OSS homepage and click
Generated Test Case:
| Test Case ID | Test Case Description | Test Steps | Expected Result | Actual Result | Pass/Fail |
| --- | --- | --- | --- | --- | --- |
|

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

Reward received: 1
Memory error during update: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 14.12 MiB is free. Process 184002 has 14.72 GiB memory in use. Of the allocated memory 14.11 GiB is allocated by PyTorch, and 493.03 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

Feedback Round 8 of 10 - Generating Integration Test Cases:
Integration Test Cases:
1. Verify that users can create an account with a valid email and password.
2. Verify that users must verify their email address within 24 hours of creating an account.
3. Verify that passwords must be at least 8 characters long, including one uppercase letter and one number.
Generated Test Case:
1. Verify that users can create an account with a va

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Reward received: 1
Memory error during update: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 14.12 MiB is free. Process 184002 has 14.72 GiB memory in use. Of the allocated memory 14.16 GiB is allocated by PyTorch, and 437.87 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

Feedback Round 9 of 10 - Generating Negative Test Cases:
Negative Test Cases:
| Test Case ID | Test Case Description | Test Steps | Expected Result | Actual Result | Pass/Fail |
|-------------|-----------------------|------------|-----------------|---------------|-----------|
| TC-01       | Verify user registration | 1. Navigate to
Generated Test Case:
| Test Case ID | Test Case Description | Test Steps | Expected Result | Actual

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Reward received: 1
Memory error during update: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 14.12 MiB is free. Process 184002 has 14.72 GiB memory in use. Of the allocated memory 14.16 GiB is allocated by PyTorch, and 437.87 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

Feedback Round 10 of 10 - Generating Exploratory Test Cases:
Exploratory Test Cases:
| Test Case ID | Test Case Description | Test Steps | Expected Result | Actual Result | Status |
| ------------ | --------------------- | ---------- | --------------- | ------------- | ------ |
| TC1 | Verify user registration | Enter valid email and password, click on register button
Generated Test Case:
| Test Case ID | Test Case Description | T